In [251]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
import json



root_url = "https://app.ticketmaster.com/discovery/v2/"

In [252]:
def desde_fecha(fecha_str):
        """
        Convierte 'YYYY-MM-DD' a 'YYYY-MM-DDTHH:MM:SSZ'
        """
        return f'{fecha_str}T00:00:00Z'

In [253]:
def hasta_fecha(fecha_str):
    """
    Convierte 'YYYY-MM-DD' a 'YYYY-MM-DDTHH:MM:SSZ' (Final del día)
    """
    return f'{fecha_str}T23:59:59Z'

In [ ]:
def extraccion_actual(ciudad, fecha_inicio, API_KEY, fecha_fin, pais='US'):

    url = f'{root_url}events.json'

    params = {
        'apikey' : API_KEY,
        'city' : ciudad,
        'countryCode' : pais,
        'startDateTime': fecha_inicio,
        'endDateTime': fecha_fin
    }

    response = requests.get(url, params=params)

    assert response.status_code == 200, "Error en la extracción"
    data = response.json()
    if '_embedded' not in data or 'events' not in data['_embedded']:
        print("No se encontraron eventos")
        return pd.DataFrame()
    
    eventos = data['_embedded']['events']
    
    df = pd.DataFrame(eventos)
    
    return df

In [255]:
API_KEY = os.getenv('TICKETMASTER_API_KEY')
assert API_KEY is not None, "Falta la variable de entorno TICKETMASTER_API_KEY"
fecha_actual = desde_fecha(datetime.now().strftime('%Y-%m-%d'))
fecha_fin = hasta_fecha(datetime.now().strftime('%Y-%m-%d'))

df = extraccion_actual('New York', fecha_actual, API_KEY, fecha_fin)
df.loc[0]

name                              Singfeld! A Musical About Nothing!
type                                                           event
id                                                  1adZZbMxZ0vZd2Fe
test                                                           False
url                https://www.ticketmaster.com/singfeld-a-musica...
locale                                                         en-us
images             [{'ratio': '16_9', 'url': 'https://s1.ticketm....
sales              {'public': {'startDateTime': '2025-11-25T18:30...
dates              {'start': {'localDate': '2026-02-12', 'localTi...
classifications    [{'primary': True, 'segment': {'id': 'KZFzniwn...
promoter           {'id': '494', 'name': 'PROMOTED BY VENUE', 'de...
promoters          [{'id': '494', 'name': 'PROMOTED BY VENUE', 'd...
pleaseNote         The Event Organizer suggests that all persons ...
products           [{'name': 'SPOTHERO THE THEATER CENTER', 'id':...
seatmap            {'staticUrl': '

In [256]:
df.columns

Index(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'sales',
       'dates', 'classifications', 'promoter', 'promoters', 'pleaseNote',
       'products', 'seatmap', 'accessibility', 'ageRestrictions', 'ticketing',
       'nameOrigin', '_links', '_embedded', 'description'],
      dtype='object')

In [257]:
df.loc[0, "sales"]

{'public': {'startDateTime': '2025-11-25T18:30:00Z',
  'startTBD': False,
  'startTBA': False,
  'endDateTime': '2026-02-12T19:00:00Z'}}

In [260]:
df = df[["name",  "dates", "_embedded"]]
df.loc[15, "dates"]

{'access': {'startDateTime': '2025-08-06T17:24:34Z',
  'startApproximate': False,
  'endDateTime': '2026-02-12T18:30:00Z',
  'endApproximate': False},
 'start': {'localDate': '2026-02-12',
  'localTime': '11:30:00',
  'dateTime': '2026-02-12T16:30:00Z',
  'dateTBD': False,
  'dateTBA': False,
  'timeTBA': False,
  'noSpecificTime': False},
 'end': {'localTime': '11:30:00',
  'dateTime': '2026-02-12T18:30:00Z',
  'approximate': False,
  'noSpecificTime': False},
 'timezone': 'America/New_York',
 'status': {'code': 'onsale'},
 'spanMultipleDays': False}

In [261]:

df['hora_inicio'] = df['dates'].apply(lambda x: x['start']['dateTime'])
df['hora_final'] = df['dates'].apply(lambda x: x['end']['dateTime'] if 'end' in x else None)
df['direccion'] = df["_embedded"].apply(lambda val: val["venues"][0]["address"]["line1"])
df['CP'] = df["_embedded"].apply(lambda val: val["venues"][0]["postalCode"])
df['parking'] = df["_embedded"].apply(lambda val: "Yes" if "parkingDetail" in val["venues"][0] else "No")
df = df.drop("dates", axis = 1)
df = df.drop("_embedded", axis=1)
df

,name,hora_inicio,hora_final,direccion,CP,parking
0,Singfeld! A Musical About Nothing!,2026-02-12T19:00:00Z,None,210 W. 50th Street,10019,Yes
1,The Banksy Museum New York!,2026-02-12T20:30:00Z,2026-02-12T22:30:00Z,277 Canal St,10013,No
2,The Banksy Museum New York!,2026-02-12T21:00:00Z,2026-02-12T23:00:00Z,277 Canal St,10013,No
3,The Banksy Museum New York!,2026-02-12T21:30:00Z,2026-02-12T23:30:00Z,277 Canal St,10013,No
4,The Banksy Museum New York!,2026-02-12T19:00:00Z,2026-02-12T21:00:00Z,277 Canal St,10013,No
5,The Banksy Museum New York!,2026-02-12T15:00:00Z,2026-02-12T17:00:00Z,277 Canal St,10013,No
6,The Banksy Museum New York!,2026-02-12T17:00:00Z,2026-02-12T19:00:00Z,277 Canal St,10013,No
7,The Banksy Museum New York!,2026-02-12T23:00:00Z,2026-02-13T01:00:00Z,277 Canal St,10013,No
8,The Banksy Museum New York!,2026-02-12T18:30:00Z,2026-02-12T20:30:00Z,277 Canal St,10013,No
9,The Banksy Museum New York!,2026-02-12T18:00:00Z,2026-02-12T20:00:00Z,277 Canal St,10013,No


In [262]:
df['hora_inicio'] = pd.to_datetime((df['hora_inicio']))
df['hora_final'] = pd.to_datetime((df['hora_final']))

In [263]:
df['hora_inicio'] = df['hora_inicio'].dt.tz_convert('America/New_York')
df['hora_final'] = df['hora_final'].dt.tz_convert('America/New_York')

In [264]:
df['hora_inicio'] = df['hora_inicio'].dt.strftime('%H:%M')
df['hora_final'] = df['hora_final'].dt.strftime('%H:%M')

In [265]:
df.head(5)

,name,hora_inicio,hora_final,direccion,CP,parking
0,Singfeld! A Musical About Nothing!,14:00,NaN,210 W. 50th Street,10019,Yes
1,The Banksy Museum New York!,15:30,17:30,277 Canal St,10013,No
2,The Banksy Museum New York!,16:00,18:00,277 Canal St,10013,No
3,The Banksy Museum New York!,16:30,18:30,277 Canal St,10013,No
4,The Banksy Museum New York!,14:00,16:00,277 Canal St,10013,No
